# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235395608992                   -0.50    7.0         
  2   -7.250369554401       -1.82       -1.40    1.0   8.95ms
  3   -7.251166928630       -3.10       -2.13    1.0   8.91ms
  4   -7.251221238545       -4.27       -2.08    2.0   10.6ms
  5   -7.251332515341       -3.95       -2.68    1.0   8.64ms
  6   -7.251338046153       -5.26       -3.13    1.0   8.78ms
  7   -7.251338725370       -6.17       -3.66    1.0   8.97ms
  8   -7.251338794101       -7.16       -4.18    2.0   10.8ms
  9   -7.251338798210       -8.39       -4.72    2.0   10.8ms
 10   -7.251338798648       -9.36       -5.16    2.0   11.0ms
 11   -7.251338798697      -10.31       -5.61    1.0   8.87ms
 12   -7.251338798704      -11.15       -6.04    2.0   10.7ms
 13   -7.251338798704      -12.14       -6.53    1.0   58.5ms
 14   -7.251338798705      -12.81       -7.16    2.0   10.9ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.060104908004922034
[ Info: Arnoldi iteration step 2: normres = 0.39039014337141453
[ Info: Arnoldi iteration step 3: normres = 1.0106381431586342
[ Info: Arnoldi iteration step 4: normres = 0.2899908596323545
[ Info: Arnoldi iteration step 5: normres = 0.4122957338342827
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (4.21e-02, 4.00e-02, 2.97e-01, 2.77e-01, 3.81e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4174810202224659
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.54e-02, 1.93e-01, 3.46e-01, 1.05e-01, 7.50e-02)
[ Info: Arnoldi iteration step 7: normres = 0.05194481170322372
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.21e-04, 1.62e-02, 4.86e-03, 2.11e-02, 3.72e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10655776470876181
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.92e